# TensorFlow - CNN (NLP) - Exercise

In [ ]:
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

### Sarcasm data

In [ ]:
with open("sarcasm.json", 'r') as f:
    datastore = json.load(f)
    
# modify train/test split 
training_size = 20000

sentences = []
labels = []

for item in datastore:
    sentences.append(item["headline"])
    labels.append(item["is_sarcastic"])
    
    
# Train/Test Split
training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]

training_labels = np.array(labels[0:training_size])
testing_labels = np.array(labels[training_size:])

## 1.) Adjust the various hyperparameters to see what's the highest validation score that you can achieve. (Try to beat .84.)

### Tokenize

In [ ]:
tokenizer = Tokenizer(num_words=10000, oov_token="<oov>")
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=32, 
                                padding="post", truncating="post")

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=32, 
                                padding="post", truncating="post")

### Create a Convolutional Neural Network

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(10000, 16, input_length=32),
    tf.keras.layers.Conv1D(128, 3, activation='relu'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(16, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.summary()

In [ ]:
model.compile(optimizer='adam', 
              loss='binary_crossentropy',
              metrics=['accuracy'])

### Train the model

In [ ]:
history = model.fit(training_padded, training_labels, validation_split=.1, epochs=5) 

### Sample headlines from the training set

In [ ]:
training_sentences[:5]

In [ ]:
training_labels[:5]

### 2.) Make up your own short headlines and test the predictions

In [ ]:
# Sarcastic headline
#my_headline = ["Man scared that neighbor is a Martian."]

# Non-sarcastic headline
my_headline = ["Congress is working on healthcare reform."]

headline_sequence = tokenizer.texts_to_sequences(my_headline)
headline_padded = pad_sequences(headline_sequence, maxlen=32, truncating="post")

#  make prediction
prediction = model.predict(headline_padded)

print(prediction)

# Output closer to 1.0 predicts sarcastic headline, closer to 0 predicts not sarcastic. 